# Lab 07 MLops

In [2]:
!nvidia-smi

Wed Nov 19 20:32:30 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.274.02             Driver Version: 535.274.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:BE:00.0 Off |                    0 |
| N/A   38C    P0              41W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Imports

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import time

## Exercise 1

### Load model

In [4]:
model_name = "sentence-transformers/multi-qa-mpnet-base-cos-v1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

### Sample input - used for experiment

In [5]:
texts = [
    "Hello World1!",
    "Java is better programming language than python - change my mind.",
    "Transformers are powerful models for NLP.",
    "Batch processing allows multiple sentences to be encoded together.",
    "This is another example input."
]

encoded_inputs = tokenizer(
    texts,
    padding=True,          
    truncation=True,      
    return_tensors="pt"    
)

input_ids = encoded_inputs["input_ids"]

for text, ids in zip(texts, input_ids):
    token_list = tokenizer.convert_ids_to_tokens(ids.tolist())
    print(f"\nTEXT: {text}\nTOKENS: {token_list}\nIDS:{ids.tolist()}")


TEXT: Hello World1!
TOKENS: ['<s>', 'hello', 'world', '##1', '!', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
IDS:[0, 7596, 2092, 2491, 1003, 2, 1, 1, 1, 1, 1, 1, 1, 1]

TEXT: Java is better programming language than python - change my mind.
TOKENS: ['<s>', 'java', 'is', 'better', 'programming', 'language', 'than', 'python', '-', 'change', 'my', 'mind', '.', '</s>']
IDS:[0, 9266, 2007, 2492, 4734, 2657, 2088, 18754, 1015, 2693, 2030, 2572, 1016, 2]

TEXT: Transformers are powerful models for NLP.
TOKENS: ['<s>', 'transformers', 'are', 'powerful', 'models', 'for', 'nl', '##p', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>']
IDS:[0, 19085, 2028, 3932, 4279, 2009, 17957, 2365, 1016, 2, 1, 1, 1, 1]

TEXT: Batch processing allows multiple sentences to be encoded together.
TOKENS: ['<s>', 'batch', 'processing', 'allows', 'multiple', 'sentences', 'to', 'be', 'encoded', 'together', '.', '</s>', '<pad>', '<pad>']
IDS:[0, 14112, 6368, 4477, 3678, 11750, 2004

In [6]:
encoded_inputs_cpu = {k: v.to("cpu") for k, v in encoded_inputs.items()}
encoded_inputs_gpu = {k: v.to("cuda") for k, v in encoded_inputs.items()}

### Measuring inference time

In [7]:
def measure_inference_time(model, encoded_inputs, runs=200):
    _ = model(**encoded_inputs)

    start_time = time.perf_counter()
    
    for _ in range(runs):
        outputs = model(**encoded_inputs)
    
    end_time = time.perf_counter()
    total_time = end_time - start_time
    return total_time / runs

#### Plain Pytorch

In [7]:
model.to(device="cpu")
plain_cpu = measure_inference_time(model, encoded_inputs_cpu)
print(f"Time Plain Torch CPU: {plain_cpu:.4f} s")

model.to(device="cuda")
plain_gpu = measure_inference_time(model, encoded_inputs_gpu)
print(f"Time Plain Torch GPU: {plain_gpu:.4f} s")

Time Plain Torch CPU: 0.2303 s
Time Plain Torch GPU: 0.0081 s


#### Eval mode

In [8]:
model.to(device="cpu")
model.eval()
eval_cpu = measure_inference_time(model, encoded_inputs_cpu)
print(f"Time Eval Torch CPU: {eval_cpu:.4f} s")

model.to(device="cuda")
model.eval()
eval_gpu = measure_inference_time(model, encoded_inputs_gpu)
print(f"Time Eval Torch GPU: {eval_gpu:.4f} s")

Time Eval Torch CPU: 0.2332 s
Time Eval Torch GPU: 0.0080 s


#### Eval mode and no grad

In [9]:
model.to(device="cpu")
model.eval()
with torch.no_grad():
    eval_no_grad_cpu = measure_inference_time(model, encoded_inputs_cpu)
    print(f"Time Eval and no Grad Torch CPU: {eval_no_grad_cpu:.4f} s")

model.to(device="cuda")
model.eval()
with torch.no_grad():
    eval_no_grad_gpu = measure_inference_time(model, encoded_inputs_gpu)
    print(f"Time Eval and no Grad Torch GPU: {eval_no_grad_gpu:.4f} s")

Time Eval and no Grad Torch CPU: 0.2250 s
Time Eval and no Grad Torch GPU: 0.0063 s


#### Full inference mode

In [10]:
model.to(device="cpu")
with torch.inference_mode():
    eval_inference_cpu = measure_inference_time(model, encoded_inputs_cpu)
    print(f"Time Inference Mode CPU: {eval_inference_cpu:.4f} s")

model.to(device="cuda")
with torch.inference_mode():
    eval_inference_gpu = measure_inference_time(model, encoded_inputs_gpu)
    print(f"Time Inference Mode GPU: {eval_inference_gpu:.4f} s")

Time Inference Mode CPU: 0.2214 s
Time Inference Mode GPU: 0.0056 s


### Result of experiment

| Method                     | CPU Time (s) | GPU Time (s) |
|----------------------------|-------------|-------------|
| Plain Torch                | 0.2303      | 0.0081      |
| Eval Torch                 | 0.2332      | 0.0080      |
| Eval Torch (no Grad)       | 0.2250      | 0.0063      |
| Inference Mode             | 0.2214      | 0.0056      |


The fastest is the inference mode. Interestingly, the Eval Torch on CPU is slower than plan torch on CPU. For the GPU Difference is small, but eval is faster.


## Exercise 2

Compiling only for GPU.

### 1. Compiling models

In [13]:
model.eval()
model.to(device="cuda")
compiled_model_gpu = torch.compile(model) 

### 2. Measuring time

In [14]:
eval_compiled_gpu = measure_inference_time(compiled_model_gpu, encoded_inputs_gpu)
print(f"Time Compiled GPU: {eval_compiled_gpu:.4f} s")

Time Compiled GPU: 0.0048 s


### 3. Speedup

In [ ]:
speed_up = 0.0081 / 0.0048
print(speed_up)

1.6875


Compiled model is faster than all of previous ones.

## Exercise 3

### 1. Ensuring model on cpu

In [20]:
model.to(device="cpu")
model.eval()

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

### 2. Quantize model

### 3. Save model to variable, veryfing

In [23]:
import torch.ao.quantization
import torch.nn as nn

model_quantized = torch.ao.quantization.quantize_dynamic(
    model,
    {nn.Linear}, 
    dtype=torch.qint8
)

print("Quantized model:")
print(model_quantized)

Quantized model:
MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (k): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (v): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (o): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (dropout): Dropout

### 4. Saving model

In [28]:
import os

torch.save(model.state_dict(), "model_original.pt")
torch.save(model_quantized.state_dict(), "model_quantized.pt")

size_original = os.path.getsize("model_original.pt")
size_quantized = os.path.getsize("model_quantized.pt")

size_original_mb = size_original / (1024 * 1024)
size_quantized_mb = size_quantized / (1024 * 1024)

print(f"Original model size: {size_original_mb:.2f} MB")
print(f"Quantized model size: {size_quantized_mb:.2f} MB")
print(f"Compression ratio: {size_original_mb / size_quantized_mb:.2f}×")

Original model size: 417.73 MB
Quantized model size: 173.10 MB
Compression ratio: 2.41×


### 5. Inference comparision

In [27]:
quantized_cpu = measure_inference_time(model_quantized, encoded_inputs_cpu)
print(f"Time Quantized CPU: {quantized_cpu:.4f} s")

no_quantized_cpu = measure_inference_time(model, encoded_inputs_cpu)
print(f"Time Non Quantized CPU: {no_quantized_cpu:.4f} s")

Time Quantized CPU: 0.0626 s
Time Non Quantized CPU: 0.2359 s


### 6. Comments on comparission

Quantized model has size of 173.10MB while original 417.73MB which is over 2.41 more. Furthermore, quantized model is almost 4 times faster than plain model.

## Exercise 4

### 1. Comparing inference

In [8]:
model.eval()
model.to(device="cuda")
compiled_model_gpu = torch.compile(model) 
compiled_model_gpu_max_autotune = torch.compile(model, mode="max-autotune") 
compiled_model_gpu_max_autotune_no_cuda_graphs = torch.compile(model, mode="max-autotune-no-cudagraphs")

In [9]:
with torch.inference_mode():
    _ = compiled_model_gpu(**encoded_inputs_gpu)
    _ = compiled_model_gpu_max_autotune(**encoded_inputs_gpu)
    _ = compiled_model_gpu_max_autotune_no_cuda_graphs(**encoded_inputs_gpu)

In [10]:
with torch.inference_mode():
    eval_compiled_gpu = measure_inference_time(compiled_model_gpu, encoded_inputs_gpu)
    print(f"Time Compiled GPU (inference_mode): {eval_compiled_gpu:.4f} s")

    eval_compiled_gpu_max_autotune = measure_inference_time(compiled_model_gpu_max_autotune, encoded_inputs_gpu)
    print(f"Time Compiled GPU Max Autotune (inference_mode): {eval_compiled_gpu_max_autotune:.4f} s")

    eval_compiled_gpu_max_autotune_no_cuda_graphs = measure_inference_time(
        compiled_model_gpu_max_autotune_no_cuda_graphs, encoded_inputs_gpu
    )
    print(f"Time Compiled GPU Max Autotune No CUDA Graphs (inference_mode): {eval_compiled_gpu_max_autotune_no_cuda_graphs:.4f} s")

Time Compiled GPU (inference_mode): 0.0031 s
Time Compiled GPU Max Autotune (inference_mode): 0.0022 s
Time Compiled GPU Max Autotune No CUDA Graphs (inference_mode): 0.0032 s


### 2. Comment on results

| Model                                    | Time |
| ---------------------------------------- | ------------- |
| Compiled GPU (default)                   | 0.0031 s      |
| Compiled GPU Max Autotune                | 0.0022 s      |
| Compiled GPU Max Autotune No CUDA Graphs | 0.0032 s      |


The fastest was Compiled GPU Max Autotune.